In [1]:
import os
import json
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [2]:
# final result (86.54%)
result_file = "../../results/scienceqa/chameleon_gpt4_test_cache.jsonl"
data_file = "../../data/scienceqa/problems.json"

In [3]:
inventory = ['knowledge_retrieval',
 'text_detector',
 'query_generator',
 'bing_search',
 'image_captioner',
 'solution_generator',
 'answer_generator']

In [4]:
prob_data = json.load(open(data_file))

with open(result_file) as f:
    results = {}
    for line in f:
        try:
            data = json.loads(line)
            results[data["pid"]] = data
        except:
            pass
        
test_pids = list(results.keys())


# construct pandas data
prob_data = pd.DataFrame(prob_data).T
res_pd = prob_data[prob_data.index.isin(test_pids)]  # test set

# update data
for index, row in res_pd.iterrows():
    result = results[index]
    
    res_pd.loc[index, 'true_false'] = result['true_false']
    
    modules = result["modules:output"]
    
    res_pd.loc[index, 'modules'] = str(modules)
    res_pd.loc[index, 'num_modules'] = int(len(modules))
    
    for module in inventory:
        res_pd.loc[index, module] = True if module in modules else False

In [5]:
res_pd

,question,choices,answer,hint,image,task,grade,subject,topic,category,...,true_false,modules,num_modules,knowledge_retrieval,text_detector,query_generator,bing_search,image_captioner,solution_generator,answer_generator
4,Which figure of speech is used in this text?\n...,"[chiasmus, apostrophe]",1,,None,closed choice,grade11,language science,figurative-language,Literary devices,...,True,"['knowledge_retrieval', 'solution_generator', ...",3.0,True,False,False,False,False,True,True
5,Which of the following could Gordon's test show?,[if the spacecraft was damaged when using a pa...,1,People can use the engineering-design process ...,image.png,closed choice,grade8,natural science,science-and-engineering-practices,Engineering practices,...,True,"['text_detector', 'knowledge_retrieval', 'solu...",4.0,True,True,False,False,False,True,True
11,What is the name of the colony shown?,"[Maryland, New Hampshire, Rhode Island, Vermont]",1,,image.png,closed choice,grade5,social science,us-history,English colonies in North America,...,False,"['text_detector', 'solution_generator', 'answe...",3.0,False,True,False,False,False,True,True
15,Would you find the word dozen on a dictionary ...,"[no, yes]",0,,None,yes or no,grade7,language science,reference-skills,Reference skills,...,False,"['knowledge_retrieval', 'solution_generator', ...",3.0,True,False,False,False,False,True,True
22,Which word would you find on a dictionary page...,"[service, stockade]",1,,None,closed choice,grade8,language science,reference-skills,Reference skills,...,False,"['knowledge_retrieval', 'solution_generator', ...",3.0,True,False,False,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21195,Which word would you find on a dictionary page...,"[president, peach]",1,,None,closed choice,grade2,language science,reference-skills,Reference skills,...,True,"['knowledge_retrieval', 'solution_generator', ...",3.0,True,False,False,False,False,True,True
21196,Which of these states is farthest west?,"[Alabama, Illinois, South Carolina, Connecticut]",1,,image.png,closed choice,grade3,social science,geography,Geography,...,False,"['text_detector', 'knowledge_retrieval', 'solu...",4.0,True,True,False,False,False,True,True
21199,Which continent is highlighted?,"[Asia, Europe, Australia, North America]",1,,image.png,closed choice,grade5,social science,geography,Oceans and continents,...,False,"['text_detector', 'solution_generator', 'answe...",3.0,False,True,False,False,False,True,True
21200,What is the direction of this push?,"[away from the bulldozer, toward the bulldozer]",0,A bulldozer clears a path for a new road. A fo...,image.png,closed choice,grade4,natural science,physics,Force and motion,...,True,"['image_captioner', 'knowledge_retrieval', 'so...",4.0,True,False,False,False,True,True,True


In [6]:
def get_num_with_contion(res_pd, key, values):
    if isinstance(values, list):
        total_pd = res_pd[res_pd[key].isin(values)]
    else:
        total_pd = res_pd[res_pd[key] == values]
        
    if len(res_pd) > 0:
        percent = "{:.2f}".format(len(total_pd) / len(res_pd) * 100)
    else:
        percent = None
    return percent

In [7]:
for module in inventory:
    use_num = get_num_with_contion(res_pd, module, True)
    nouse_num = get_num_with_contion(res_pd, module, False)
    print(module, use_num, nouse_num)

knowledge_retrieval 81.30 18.70
text_detector 29.64 70.36
query_generator 11.15 88.85
bing_search 11.27 88.73
image_captioner 6.48 93.52
solution_generator 100.00 0.00
answer_generator 100.00 0.00
